In [1]:
lista_uhe = []

usina = {
    "Nome": "UHE DO MARCATO", # Nome da Usina
    "Vmax": 100.,             # Volume Máximo em hm^3
    "Vmin": 20.,              # Volume Mínimo em hm^3
    "VI": 70,                # Volume Inicial em hm^3
    "Prod": 0.95,             # Produtibilidade em MWmed/hm^3
    "Engol": 60.,             # Engolimento Máximo em hm^3
    "Afl": [                  # Cenários de Afluências (linha: Estágio, coluna: cenário)
            [ 23, 16],
            [ 19, 14],
            [ 15, 11]
    ]
}

lista_uhe.append(usina)

#
# Retirar os comentários abaixo para considerar 2 UHEs
#
#usina = {
#    "Nome": "UHE DO VASCAO",
#    "Vmax": 200.,
#    "Vmin": 40.,
#    "VI": 200,
#    "Prod": 0.85,
#    "Engol": 100.,
#    "Afl": [
#            [ 46, 32],
#            [ 38, 28],
#            [ 30, 22]
#    ]
#}
#lista_uhe.append(usina)

usina = {
    "Nome": "GT_1",     # Nome da Usina Térmica 1
    "Capac": 15.,       # Capacidade Máxima de Geração MWMed
    "Custo": 10.        # Custo de Operação $/MWMed
}

lista_ute = []

lista_ute.append(usina)

usina = {
    "Nome": "GT_2",     # Nome da Usina Térmica 2
    "Capac": 10.,       # Capacidade Máxima de Geração MWmed
    "Custo": 25.        # Custo de Operação $/MWMed
}

lista_ute.append(usina)

#
# d_gerais para o caso 1 UHE
#
d_gerais = {
    "CDef": 500.,             # Custo de Déficit $/MWMed
    "Carga": [ 50., 50., 50.], # Lista com carga a ser atendida por estágio
    "Nr_Disc": 12,            # Número de Discretizações
    "Nr_Est": 3,              # Número de Estágios
    "Nr_Cen": 2               # Número de Cenários de Afluências
}

#
# d_gerais para o caso 2 UHE (Comentar o bloco acima e descomentar o bloco abaixo)
#
#d_gerais = {
#    "CDef": 500.,
#    "Carga": [ 100, 100., 100],
#    "Nr_Disc": 3,
#    "Nr_Est": 3,
#    "Nr_Cen": 2
#}

#
# Cria dicionário de dados com todas as informações do sistema em estudo
#
sistema = {
    "DGer": d_gerais,
    "UHE": lista_uhe,
    "UTE": lista_ute
}

In [374]:
from cvxopt.modeling import matrix, solvers, spmatrix
from cvxopt.modeling import op
from cvxopt import printing
import numpy as np

def pl_unico(sistema, cenario, imprime, Num_UHE, Num_UTE):

    # vf = []
    # vt = []
    # vv = []
    # for i,iusi in enumerate(sistema["UHE"]):
    #     vf.append(variable(sistema["DGer"]["Nr_Est"], "Volume Final na Usina "+iusi["Nome"]))
    #     vt.append(variable(sistema["DGer"]["Nr_Est"], "Volume Turbinado na Usina "+iusi["Nome"]))
    #     vv.append(variable(sistema["DGer"]["Nr_Est"], "Volume Vertido na Usina "+iusi["Nome"]))
    # gt = []
    # for i,iusi in enumerate(sistema["UTE"]):
    #     gt.append(variable(sistema["DGer"]["Nr_Est"], "Geração na Usina Térmica "+iusi["Nome"]))
    # deficit = variable(sistema["DGer"]["Nr_Est"], "Déficit de Energia no Sistema")



    C1 = np.zeros(((3*Num_UHE)+Num_UTE+1))   

    for i, iusi in enumerate(sistema["UHE"]):
        C1[(3*(i+1))-1] = 0.01

    for i, iusi in enumerate(sistema["UTE"]):

        C1[3*Num_UHE+i] = iusi['Custo']

    C1[3*Num_UHE+Num_UTE] = sistema["DGer"]["CDef"]

    
    C = []

    for i in range(0,sistema["DGer"]["Nr_Est"]):
        C.extend( C1)


    C = matrix(C)


    A1 = np.zeros((Num_UHE+1,((3*Num_UHE)+Num_UTE+1))) 
    A2 = np.zeros((Num_UHE+1,((3*Num_UHE)+Num_UTE+1))) 

    for i, iusi in enumerate(sistema["UHE"]):
        A1[-1,1+(i*3)] = iusi["Prod"]
        # A1[1,1+(i*3)] = -iusi["Prod"]
        for j in range(0,3):
            A1[i,j+(i*3)] = 1
            # A1[0,j+(i*3)] = -1

    for i, iusi in enumerate(sistema["UTE"]):
        A1[-1,3*Num_UHE+i] = 1
        # A1[1,3*Num_UHE+i] = -1

    A1[1,-1] = 1
    # A1[1,-1] = -1

    # A = []

    A = np.vstack((np.hstack((A1, A2)), np.hstack((A2, A1))))

    for i in range(2,sistema["DGer"]["Nr_Est"]):

        A2 = np.zeros(((Num_UHE+1)*i,((3*Num_UHE)+Num_UTE+1))) 

        A3 = np.zeros((Num_UHE+1,((3*Num_UHE)+Num_UTE+1)*i)) 

        A = np.vstack((np.hstack((A, A2)), np.hstack((A3, A1))))

    for i in range(0,sistema["DGer"]["Nr_Est"]-1):

        #################################################################################################################################### Aqui #################################
        A[2+(i*2),i*((3*Num_UHE)+Num_UTE+1)] = -1
        # A[2+(i*2),i*((3*Num_UHE)+Num_UTE+1)] = 1

    # A.append(A)
    # print(A)
    # A = np.transpose(A)
    # print(A)

    # A = matrix(A,(((3*Num_UHE)+Num_UTE+1)*sistema["DGer"]["Nr_Est"],(Num_UHE+1)*sistema["DGer"]["Nr_Est"]),'d')
    # A = matrix(A)
    


    A = matrix(A,((Num_UHE+1)*sistema["DGer"]["Nr_Est"],((3*Num_UHE)+Num_UTE+1)*sistema["DGer"]["Nr_Est"]),'d')
    # print(A.trans())
    # A = A.trans()

    printing.options['width'] = -1
    # print(A)

    b = np.zeros(((Num_UHE)+1)*sistema["DGer"]["Nr_Est"]) 

    for i in range(0,sistema["DGer"]["Nr_Est"]):

        if i == 0:

            for j, iusi in enumerate(sistema["UHE"]):
                
                b[j] = iusi["VI"]+iusi["Afl"][i][cenario]

            b[(Num_UHE)] = sistema["DGer"]["Carga"][i]

        else:

            for j, iusi in enumerate(sistema["UHE"]):
                
                b[j+(((Num_UHE)+1)*i)] = iusi["Afl"][i][cenario]
            
            b[Num_UHE+(((Num_UHE)+1)*i)] = sistema["DGer"]["Carga"][i]

    b = matrix(b)

    G1 = np.identity(((3*Num_UHE)+Num_UTE+1)*sistema["DGer"]["Nr_Est"])

    G2 = np.identity(((3*Num_UHE)+Num_UTE+1)*sistema["DGer"]["Nr_Est"])

    
    G = np.vstack((G1, -G2))
    G = matrix(G,(((3*Num_UHE)+Num_UTE+1)*sistema["DGer"]["Nr_Est"]*2,((3*Num_UHE)+Num_UTE+1)*sistema["DGer"]["Nr_Est"]),'d')
    # print(G)
    # G = matrix(np.vstack((G1, -G2)),(((3*Num_UHE)+Num_UTE+1)*sistema["DGer"]["Nr_Est"]*2,((3*Num_UHE)+Num_UTE+1)*sistema["DGer"]["Nr_Est"]),'d')

    print(G)
    # G = G.trans()
    # print(G)
    



    h = np.zeros(((3*Num_UHE)+Num_UTE+1)*sistema["DGer"]["Nr_Est"]*2) 

    for j in range(0,sistema["DGer"]["Nr_Est"]):

        for i, iusi in enumerate(sistema["UHE"]):

            h[(i*3)+(((3*Num_UHE)+Num_UTE+1)*j)] = iusi["Vmax"]
            h[(i*3)+1+(((3*Num_UHE)+Num_UTE+1)*j)] = iusi["Engol"]
            h[(i*3)+2+(((3*Num_UHE)+Num_UTE+1)*j)] = 10**10

            h[(i*3)+(((3*Num_UHE)+Num_UTE+1)*j)+((3*Num_UHE)+Num_UTE+1)*sistema["DGer"]["Nr_Est"]] = -iusi["Vmin"]
            h[(i*3)+1+(((3*Num_UHE)+Num_UTE+1)*j)+((3*Num_UHE)+Num_UTE+1)*sistema["DGer"]["Nr_Est"]] = 0
            h[(i*3)+2+(((3*Num_UHE)+Num_UTE+1)*j)+((3*Num_UHE)+Num_UTE+1)*sistema["DGer"]["Nr_Est"]] = 0


        for i, iusi in enumerate(sistema["UTE"]):

            h[(3*Num_UHE)+i+(((3*Num_UHE)+Num_UTE+1)*j)] = iusi["Capac"]
            h[(3*Num_UHE)+i+(((3*Num_UHE)+Num_UTE+1)*j)+((3*Num_UHE)+Num_UTE+1)*sistema["DGer"]["Nr_Est"]] = 0
        
        h[(3*Num_UHE)+Num_UTE+(((3*Num_UHE)+Num_UTE+1)*j)] = sistema["DGer"]["Carga"][j]
        h[(3*Num_UHE)+Num_UTE+(((3*Num_UHE)+Num_UTE+1)*j)+((3*Num_UHE)+Num_UTE+1)*sistema["DGer"]["Nr_Est"]] = 0


    h = matrix(h)
    print(h)

    

    sol = solvers.lp(C, G, h ,A,b)
    # sol = solvers.lp(C, A,b,G,h )
    print(sol['x'])
    # print(A)
    # print(A1)
    # print(A2)
    # print(A3)
        


# cvxopt.solvers.options['show_progress']

In [375]:
Num_UHE = len(sistema["UHE"])

Num_UTE = len(sistema["UTE"])
imprime = 0

resultado = pl_unico(sistema, 0, imprime,Num_UHE,Num_UTE)

# A = np.zeros((2,((3*Num_UHE)+Num_UTE+1)))  
# print(A)


# f = np.identity(3)
# j = np.identity(3)
# # j = spmatrix(-1.0, range(3), range(3))
# print(matrix(np.vstack((f, -j))))
# print(j)

[ 1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
[ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00  0.00e+00